In [2]:
import os
import sys

current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from apikey import apikey, langchain_apikey

os.environ['OPENAI_API_KEY'] = apikey
os.environ['LANGCHAIN_API_KEY'] = langchain_apikey
os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [3]:
# To run the example below, ensure the environment variable OPENAI_API_KEY is set
import openai

from langsmith.run_trees import RunTree

In [5]:
# This can be a user input to your app
question = "Can you summarize this morning's meetings?"


In [6]:
# Create a top-level run
pipeline = RunTree(
    name="Chat Pipeline",
    run_type="chain",
    inputs={"question": question}
)

# cc： 核心看来是在这个地方，通过RunTree 来进行“轨迹跟踪”，得到一个pipeline的对象。

In [7]:
pipeline

RunTree(id=UUID('53115abc-2476-4cff-87f2-7e4c099b68df'), name='Chat Pipeline', start_time=datetime.datetime(2024, 3, 18, 7, 49, 57, 368447, tzinfo=datetime.timezone.utc), run_type='chain', end_time=None, extra={}, error=None, serialized={'name': 'Chat Pipeline'}, events=None, inputs={'question': "Can you summarize this morning's meetings?"}, outputs=None, reference_example_id=None, parent_run_id=None, tags=None, parent_run=None, child_runs=[], session_name='default', session_id=None, client=Client (API URL: https://api.smith.langchain.com), dotted_order='20240318T074957368447Z53115abc-2476-4cff-87f2-7e4c099b68df', trace_id=UUID('53115abc-2476-4cff-87f2-7e4c099b68df'))

In [9]:

# This can be retrieved in a retrieval step
context = "During this morning's meeting, we solved all world conflict."

messages = [
    { "role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context." },
    { "role": "user", "content": f"Question: {question}\nContext: {context}"}
]

messages

[{'role': 'system',
  'content': "You are a helpful assistant. Please respond to the user's request only based on the given context."},
 {'role': 'user',
  'content': "Question: Can you summarize this morning's meetings?\nContext: During this morning's meeting, we solved all world conflict."}]

In [10]:
# Create a child run
child_llm_run = pipeline.create_child(
    name="OpenAI Call",
    run_type="llm",
    inputs={"messages": messages},
)

child_llm_run

# cc：这里就可以通过上文创建的pipeline对象来进行子链路创建。
# 这只是一个例子，后面应该有更自动化的方式来进行子链路创建。

RunTree(id=UUID('a6959c1f-1dcb-41c0-a50e-84493550dfee'), name='OpenAI Call', start_time=datetime.datetime(2024, 3, 18, 7, 51, 39, 322991, tzinfo=datetime.timezone.utc), run_type='llm', end_time=None, extra={}, error=None, serialized={'name': 'OpenAI Call'}, events=None, inputs={'messages': [{'role': 'system', 'content': "You are a helpful assistant. Please respond to the user's request only based on the given context."}, {'role': 'user', 'content': "Question: Can you summarize this morning's meetings?\nContext: During this morning's meeting, we solved all world conflict."}]}, outputs={}, reference_example_id=None, parent_run_id=UUID('53115abc-2476-4cff-87f2-7e4c099b68df'), tags=None, parent_run=RunTree(id=UUID('53115abc-2476-4cff-87f2-7e4c099b68df'), name='Chat Pipeline', start_time=datetime.datetime(2024, 3, 18, 7, 49, 57, 368447, tzinfo=datetime.timezone.utc), run_type='chain', end_time=None, extra={}, error=None, serialized={'name': 'Chat Pipeline'}, events=None, inputs={'question':

In [11]:
# Generate a completion
client = openai.Client()
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages
)

chat_completion

ChatCompletion(id='chatcmpl-9425L5j9wDQXvgYljHecDmJxEXyla', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The meeting this morning achieved a significant milestone by resolving all world conflicts.', role='assistant', function_call=None, tool_calls=None))], created=1710748355, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f2ebda25a', usage=CompletionUsage(completion_tokens=14, prompt_tokens=55, total_tokens=69))

In [12]:
# End the runs and log them
child_llm_run.end(outputs=chat_completion)
child_llm_run.post()

pipeline.end(outputs={"answer": chat_completion.choices[0].message.content})
pipeline.post()